# CreateView

### Example

[source](https://github.com/LCOGT/valhalla/blob/c9abea8200e389dbabd3ff31e4ef17e1ae02bd74/valhalla/sciapplications/views.py#L27)


```python

class SciApplicationCreateView(LoginRequiredMixin, CreateView):
    template_name = 'sciapplications/create.html'
    model = ScienceApplication

    def get_form_class(self):
        try:
            return FORM_CLASSES[self.call.proposal_type]
        except KeyError:
            raise Http404

    def get_success_url(self):
        if self.object.status == ScienceApplication.DRAFT:
            messages.add_message(self.request, messages.SUCCESS, _('Application created'))
            return reverse('sciapplications:update', kwargs={'pk': self.object.id})
        else:
            messages.add_message(self.request, messages.SUCCESS, _('Application successfully submitted'))
            return reverse('sciapplications:index')

    def get_initial(self):
        return {'call': self.call}

    def get(self, request, *args, **kwargs):
        self.object = None
        self.call = get_object_or_404(Call, pk=kwargs['call'])
        form = self.get_form()
        timerequest_form = timerequest_formset()
        return self.render_to_response(
            self.get_context_data(form=form, timerequest_form=timerequest_form, call=self.call)
        )

    def post(self, request, *args, **kwargs):
        self.object = None
        self.call = get_object_or_404(Call, pk=kwargs['call'])
        form = self.get_form()
        form.instance.submitter = request.user
        timerequest_form = timerequest_formset(self.request.POST)
        if form.is_valid() and timerequest_form.is_valid():
            return self.forms_valid({'main': form, 'tr': timerequest_form})
        else:
            return self.forms_invalid({'main': form, 'tr': timerequest_form})

    def forms_valid(self, forms):
        self.object = forms['main'].save()
        forms['tr'].instance = self.object
        forms['tr'].save()
        return HttpResponseRedirect(self.get_success_url())

    def forms_invalid(self, forms):
        return self.render_to_response(
            self.get_context_data(form=forms['main'], timerequest_form=forms['tr'], call=self.call)
        )

        
```